In [20]:
!pip install --quiet google-cloud-bigquery google-cloud-aiplatform --quiet

In [56]:
from google.cloud import bigquery
import vertexai

PROJECT_ID = "qwiklabs-gcp-04-c85702fbf940"
LOCATION = "global"
EMBEDDED_TABLE_ID = f"{PROJECT_ID}.challenge2.faq_embedded"
EMBEDDED_MODEL_ID = f"{PROJECT_ID}.challenge2.Embeddings"

vertexai.init(project=PROJECT_ID, location=LOCATION)
bq_client = bigquery.Client(project=PROJECT_ID)

def query_vector_db(question):
    query = f"""
    SELECT
     query.query,
    base.content
    FROM VECTOR_SEARCH(
      TABLE `{EMBEDDED_TABLE_ID}`,
      'ml_generate_embedding_result',
      (
        SELECT
          ml_generate_embedding_result AS embedding,
          content AS query
        FROM ML.GENERATE_EMBEDDING(
          MODEL `{EMBEDDED_MODEL_ID}`,
          (SELECT '{question}' AS content)
        )
      ),
      top_k => 3,
      options => '{{"fraction_lists_to_search": 1.0}}'
    ) AS result
    """
    return bq_client.query(query).to_dataframe()


In [58]:
from vertexai.preview.generative_models import GenerativeModel
from vertexai.preview.generative_models import GenerativeModel, ChatSession, SafetySetting

system_instruction = """
ROLE:
You are a FAQ chat bot

TASK:
Answer questions based on the given context

INSTRUCTIONS:
- Do not answer any question unrelated to the given context
- Give only the answer and do not explain anything.
- Respond 'I can't help with It' if asked about anything that you are not allowed to respond even part of it has valid query
"""

model = GenerativeModel(
    model_name="gemini-2.0-flash-001",
    system_instruction=system_instruction,
    safety_settings=[
        SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="BLOCK_LOW_AND_ABOVE"),
        SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="BLOCK_LOW_AND_ABOVE"),
        SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="BLOCK_LOW_AND_ABOVE"),
        SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="BLOCK_LOW_AND_ABOVE"),
    ]
)
chat = model.start_chat()

def chat_bot(prompt):
    results = query_vector_db(prompt)
    context = "\n\n".join([f"{row['content']}" for _,row in results.iterrows()])
    prompt = f"You are a faq service. Use the context:{context} to answer: {prompt}"
    code = chat.send_message(prompt)
    return code.text

def chat_user():
    while True:
        prompt = input("User: ")
        if prompt.lower() in ["exit", "quit"]:
            print("Chat closed.")
            break
        print("FAQ assistant:", chat_bot(prompt))

chat_user()

User: What is the population of Aurora Bay?
FAQ assistant: Aurora Bay has a population of approximately 3,200 residents, although it can fluctuate seasonally due to temporary fishing and tourism workforces.

User: When was it founded
FAQ assistant: Aurora Bay was founded in 1901 by a group of fur traders who recognized the region’s strategic coastal location.

User: exit
Chat closed.
